In [2]:
import pandas as pd, numpy as np, seaborn as sns,matplotlib.pyplot as plt,shap,pickle,basketball_functions as bb
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split, cross_val_score,cross_validate, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sportsreference.ncaab.teams import Teams
from sportsreference.ncaab.teams import Team
from sportsreference.ncaab.roster import Player
from tqdm import tqdm
from sportsreference.ncaab.boxscore import Boxscore
from sklearn.linear_model import LogisticRegression,LinearRegression
import xgboost as xgb
from pprint import pprint
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from kenpompy.utils import login
import kenpompy.summary as kp
import kenpompy.misc as kpm

In [13]:
df = pd.read_csv("ml_df_l4.csv")
df = df.drop(columns = [df.columns[i] for i in [6,39,66,67,68,69,70,73,74,71]])
print("Assuring features are scaled appropriately")
for i in df.loc[:,df.columns[df.columns.str.contains('percent')]].columns:
    if df[i].max() > 1:
        df[i] = [j/100 for j in df[i]]
df.sample(5)

Assuring features are scaled appropriately


,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,offensive_rating,offensive_rebound_percentage,allowed_assist_percentage,allowed_block_percentage,allowed_effective_field_goal_percentage,allowed_field_goal_percentage,allowed_free_throw_attempt_rate,allowed_free_throw_percentage,allowed_offensive_rebound_percentage,allowed_steal_percentage,allowed_three_point_attempt_rate,allowed_three_point_field_goal_percentage,allowed_two_point_field_goal_percentage,allowed_total_rebound_percentage,allowed_true_shooting_percentage,allowed_turnover_percentage,pace,simple_rating_system,steal_percentage,three_point_attempt_rate,three_point_field_goal_percentage,two_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,win_percentage,opp_assist_percentage,opp_block_percentage,opp_effective_field_goal_percentage,opp_field_goal_percentage,opp_free_throw_attempt_rate,opp_free_throw_percentage,opp_offensive_rating,opp_offensive_rebound_percentage,opp_pace,opp_simple_rating_system,opp_steal_percentage,opp_three_point_attempt_rate,opp_three_point_field_goal_percentage,opp_two_point_field_goal_percentage,opp_total_rebound_percentage,opp_true_shooting_percentage,opp_turnover_percentage,opp_win_percentage,opp_assist_percentage_allowed,opp_block_percentage_allowed,opp_effective_field_goal_percentage_allowed,opp_field_goal_percentage_allowed,opp_free_throw_attempt_rate_allowed,opp_free_throw_percentage_allowed,opp_offensive_rebound_percentage_allowed,opp_steal_percentage_allowed,opp_three_point_attempt_rate_allowed,opp_three_point_field_goal_percentage_allowed,opp_two_point_field_goal_percentage_allowed,opp_total_rebound_percentage_allowed,opp_true_shooting_percentage_allowed,opp_turnover_percentage_allowed,result,AdjO,AdjD,AdjT,Luck,OppO,OppD
12814,0.491,0.129,0.500,0.433,0.438,0.674,108.2,0.353,0.456,0.093,0.471,0.413,0.339,0.717,0.274,0.093,0.363,0.325,0.462,0.463,0.511,0.164,70.4,15.04,0.089,0.441,0.306,0.533,0.537,0.536,0.150,0.806,0.516,0.104,0.522,0.437,0.369,0.693,106.0,0.298,76.1,11.12,0.082,0.489,0.349,0.521,0.497,0.553,0.167,0.516,0.434,0.124,0.479,0.425,0.359,0.741,0.308,0.110,0.347,0.312,0.484,0.503,0.523,0.157,1,111.4,95.5,69.2,0.103,107.0,100.3
5243,0.535,0.082,0.464,0.413,0.391,0.708,102.2,0.331,0.553,0.133,0.481,0.436,0.321,0.729,0.325,0.089,0.274,0.331,0.475,0.500,0.519,0.151,65.2,4.10,0.084,0.312,0.325,0.453,0.500,0.508,0.148,0.484,0.486,0.114,0.476,0.421,0.361,0.687,104.1,0.335,58.0,7.60,0.083,0.341,0.323,0.472,0.519,0.512,0.145,0.515,0.537,0.079,0.470,0.406,0.312,0.661,0.304,0.083,0.364,0.355,0.435,0.481,0.499,0.140,0,106.3,100.4,62.7,0.002,108.0,102.1
51895,0.504,0.115,0.513,0.458,0.298,0.660,104.0,0.292,0.508,0.071,0.481,0.423,0.332,0.733,0.280,0.084,0.331,0.351,0.459,0.496,0.521,0.187,70.9,-5.08,0.107,0.324,0.342,0.513,0.504,0.536,0.153,0.531,0.463,0.075,0.474,0.425,0.381,0.700,98.3,0.306,75.0,-10.16,0.095,0.313,0.319,0.472,0.505,0.515,0.176,0.533,0.502,0.072,0.471,0.424,0.387,0.656,0.297,0.094,0.346,0.276,0.501,0.495,0.505,0.180,0,98.2,104.5,69.1,-0.050,99.0,107.7
39877,0.610,0.119,0.538,0.465,0.436,0.690,108.9,0.258,0.553,0.106,0.477,0.413,0.349,0.707,0.276,0.062,0.394,0.325,0.470,0.503,0.515,0.188,69.4,12.98,0.112,0.381,0.385,0.515,0.497,0.571,0.150,0.750,0.556,0.106,0.510,0.461,0.364,0.705,106.8,0.312,69.0,11.57,0.107,0.294,0.331,0.515,0.518,0.544,0.153,0.743,0.464,0.089,0.475,0.422,0.409,0.697,0.273,0.078,0.315,0.338,0.460,0.482,0.517,0.185,1,111.9,96.3,68.4,-0.021,105.9,102.3
35723,0.567,0.064,0.546,0.462,0.278,0.738,111.9,0.247,0.527,0.099,0.511,0.448,0.268,0.725,0.263,0.061,0.363,0.346,0.506,0.508,0.539,0.161,65.7,13.75,0.086,0.440,0.380,0.527,0.492,0.573,0.131,0.639,0.440,0.125,0.466,0.407,0.345,0.696,98.7,0.287,66.2,3.17,0.078,0.370,0.317,0.459,0.490,0.503,0.147,0.500,0.517,0.092,0.488,0.424,0.425,0.709,0.295,0.086,0.353,0.366,0.455,0.510,0.531,0.152,1,114.6,100.5,65.8,0.013,110.1,101.1


# Outlier Removal

In [14]:
idx_to_remove = []
for i in df.columns.tolist():
    stat,p = stats.normaltest(df[i])
    if p <= 0.05:
        zscores = stats.zscore(df[i])
        zscores = np.abs(zscores)
        idx_list = []
        for idx,i in enumerate(zscores):
            if i >=3.2:
                idx_list.append(idx)
        for i in idx_list:
            if i not in idx_to_remove:
                idx_to_remove.append(i)
print("Identified {} outliers to remove".format(len(idx_to_remove)))
df2 = df.drop(idx_to_remove,axis = 0)

Identified 5457 outliers to remove


# Eliminating Multicolinearity

In [ ]:
corr=df2.corr()
high_corr_var=np.where(corr>0.8)
high_corr_var=[(corr.columns[x],corr.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
#for tup in high_corr_var:
    #print('Removing {} as it is highly correllated with {}'.format(tup[1],tup[0]))

to_drop = []
for x,y in high_corr_var:
    if y not in to_drop:
        to_drop.append(y)

df2 = df2.drop([i for i in to_drop],axis = 1)
X = df2.drop('result',axis = 1)
y = df2.result
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify = y)
neutral_court_cols = X_train.columns.tolist()
X_train
df2.sample(5)

In [19]:
clf = LogisticRegression(max_iter = 10000)
clf.fit(X_train,y_train)
print("Training Accuracy")
print(cross_val_score(clf,X_train,y_train))
print("Testing accuracy")
print(cross_val_score(clf,X_test,y_test))
print("Training ROC")
print(cross_val_score(clf,X_train,y_train,scoring = 'roc_auc'))
print("Testing accuracy")
print(cross_val_score(clf,X_test,y_test,scoring = 'roc_auc'))
score2 = accuracy_score(y_test,clf.predict(X_test))

Training Accuracy
[0.74198212 0.76114105 0.73971342 0.74786381 0.75575128]
Testing accuracy
[0.74566246 0.7626183  0.73462145 0.73264984 0.75424063]
Training ROC
[0.82948725 0.84511905 0.82337501 0.83726202 0.8384951 ]
Testing accuracy
[0.83629096 0.84090492 0.82213561 0.81885207 0.8400998 ]


In [22]:
import statsmodels.api as sm
X2 = sm.add_constant(X_train)
est = sm.Logit(y_train, X2)
est2 = est.fit(maxiter = 10000)
print(est2.summary())

         Current function value: inf
         Iterations: 10000
                           Logit Regression Results                           
Dep. Variable:                 result   No. Observations:                38036
Model:                          Logit   Df Residuals:                    37979
Method:                           MLE   Df Model:                           56
Date:                Thu, 03 Mar 2022   Pseudo R-squ.:                    -inf
Time:                        22:18:29   Log-Likelihood:                   -inf
converged:                      False   LL-Null:                       -26353.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                                                    coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                          3.287e+06        nan        n

Maximum Likelihood optimization failed to converge. Check mle_retvals
invalid value encountered in sqrt
invalid value encountered in greater
invalid value encountered in less
invalid value encountered in less_equal


In [24]:
pvals = est2.pvalues.to_frame('p_val').reset_index().rename(columns = {'index':'coef'})
pvals['p_val'] = round(pvals.p_val,3)
sig_vars = pvals[pvals.p_val <= 0.05]['coef'].tolist()
X_train = X_train[sig_vars]
X_test = X_test[sig_vars]
clf.fit(X_train,y_train)
print("Training Accuracy")
print(cross_val_score(clf,X_train,y_train))
print("Testing accuracy")
print(cross_val_score(clf,X_test,y_test))
print("Training ROC")
print(cross_val_score(clf,X_train,y_train,scoring = 'roc_auc'))
print("Testing accuracy")
print(cross_val_score(clf,X_test,y_test,scoring = 'roc_auc'))

Training Accuracy
[0.74342797 0.75930064 0.74050217 0.74760089 0.75640857]
Testing accuracy
[0.74842271 0.76301262 0.73501577 0.73304416 0.75424063]
Training ROC
[0.82936839 0.8438481  0.82333191 0.83638913 0.83753042]
Testing accuracy
[0.8356064  0.84103561 0.82114607 0.81853155 0.83987309]


In [26]:
df_agg = bb.create_current_dataframe()
kp_df = bb.create_kp_df()
print("Done")

Done


In [69]:
get_game_prediction_neutral(df_agg,kp_df,"Purdue",'Michigan State')

0.72

In [67]:
def get_game_prediction_neutral(df_agg,kp_df,team_1,team_2):

# Win probability is expressed in the form of team_1 beating team_2 on a neutral court

    class CustomError(Exception):
        pass

    if team_1 in df_agg.name.unique().tolist() and team_2 in df_agg.name.unique().tolist():
        temp_df_2 = df_agg.loc[df_agg.name == team_1].loc[:,df_agg.columns[df_agg.columns.str.contains('percen') | df_agg.columns.str.contains('rat') | (df_agg.columns == 'pace') | (df_agg.columns == 'name')]]
        temp_df_3 = df_agg.loc[df_agg.name == team_2].loc[:,df_agg.columns[df_agg.columns.str.contains('percen') | df_agg.columns.str.contains('rat') | (df_agg.columns == 'pace') | (df_agg.columns == 'name')]]
        old_names = temp_df_2.columns[temp_df_2.columns.str.contains('opp_')].to_list()
        new_names = [i.replace('opp','allowed') for i in temp_df_2.columns[temp_df_2.columns.str.contains('opp_')].to_list()]
        name_dict = dict(zip(old_names,new_names))
        temp_df_2_dict = temp_df_2.rename(columns = name_dict).to_dict(orient = 'list')
        temp_df_2_dict.update(temp_df_3.iloc[0,:10].rename(dict(zip(temp_df_3.iloc[0,:10].index,['opp_'+str(i) for i in temp_df_3.iloc[0,:10].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_2_dict.update(temp_df_3.iloc[0,25:].rename(dict(zip(temp_df_3.iloc[0,25:].index,['opp_'+str(i) for i in temp_df_3.iloc[0,25:].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_2_dict.update(temp_df_3.iloc[0,10:25].rename(dict(zip(temp_df_3.iloc[0,10:25].index,[i + "_allowed" for i in temp_df_3.iloc[0,10:25].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_3_dict = temp_df_3.rename(columns = name_dict).to_dict(orient = 'list')
        temp_df_3_dict.update(temp_df_2.iloc[0,:10].rename(dict(zip(temp_df_2.iloc[0,:10].index,['opp_'+str(i) for i in temp_df_2.iloc[0,:10].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_3_dict.update(temp_df_2.iloc[0,25:].rename(dict(zip(temp_df_2.iloc[0,25:].index,['opp_'+str(i) for i in temp_df_2.iloc[0,25:].index]))).to_frame().transpose().to_dict(orient = 'list'))
        temp_df_3_dict.update(temp_df_2.iloc[0,10:25].rename(dict(zip(temp_df_2.iloc[0,10:25].index,[i + "_allowed" for i in temp_df_2.iloc[0,10:25].index]))).to_frame().transpose().to_dict(orient = 'list'))
        for i in temp_df_3_dict.keys():
            temp_df_2_dict[i]+=(temp_df_3_dict[i])
        master_dict = {key:[] for key in temp_df_2_dict.keys()}

        for i in temp_df_2_dict.keys():
            master_dict[i] += temp_df_2_dict[i]
        df_pred = pd.DataFrame.from_dict(master_dict)
        df_pred = df_pred[df_pred.name == team_1]
        for i in df_pred.loc[:,df_pred.columns[df_pred.columns.str.contains('percent')]].columns:
             if df_pred[i].max() > 1:
                df_pred[i] = [j/100 for j in df_pred[i]]
        df_pred = pd.merge(df_pred,kp_df,left_on = 'name',right_on = "Team",how = 'left').drop("Team",axis = 1)
        return round(clf.predict_proba(df_pred[neutral_court_cols])[0][1],2)
    else:
        raise CustomError("Check Team Names")

In [70]:
with open('neutral_court_model.pickle','wb') as f:
    pickle.dump(clf,f)

In [ ]:
sum_df = kp.get_efficiency(browser,2022)

In [86]:
def get_expected_spread(sum_df,team_1,team_2,verbose = False,neutral = True):

    for i in sum_df.columns:
        try:
            sum_df[i] = pd.to_numeric(sum_df[i])
        except:
            pass
    sum_df.columns = ['_'.join(i.split()) for i in sum_df.columns.tolist()]
    sum_df.columns = [i.replace(".","") for i in sum_df.columns]
    sum_df.columns = [i.replace("-","_") for i in sum_df.columns]
    off_eff_avg = round(sum_df.Off_Efficiency_Adj.mean(),2)
    def_eff_avg = round(sum_df.Def_Efficiency_Adj.mean(),2)

    team_1_off_eff = sum_df.loc[sum_df.Team == team_1].Off_Efficiency_Adj.item()
    team_1_def_eff = sum_df.loc[sum_df.Team == team_1].Def_Efficiency_Adj.item()
    team_2_off_eff = sum_df.loc[sum_df.Team == team_2].Off_Efficiency_Adj.item()
    team_2_def_eff = sum_df.loc[sum_df.Team == team_2].Def_Efficiency_Adj.item()
    team_1_tempo = sum_df.loc[sum_df.Team == team_1].Tempo_Adj.item()
    team_2_tempo = sum_df.loc[sum_df.Team == team_2].Tempo_Adj.item()

    team_1_off_pct = (team_1_off_eff/off_eff_avg)-1
    team_1_off_pct_adjustment_factor = 1+round(team_1_off_pct + ((team_2_def_eff/def_eff_avg) - 1),2)
    team_1_off_adj = round(off_eff_avg * team_1_off_pct_adjustment_factor,2)
    

    team_2_off_pct = (team_2_off_eff/off_eff_avg)-1
    team_2_off_pct_adjustment_factor = 1+round(team_2_off_pct + ((team_1_def_eff/def_eff_avg) - 1),2)
    team_2_off_adj = round(off_eff_avg * team_2_off_pct_adjustment_factor,2)
    if verbose:
        if neutral:
            print(f"{team_1}'s expected efficiency is {team_1_off_adj}")
            print(f"{team_2}'s expected efficiency is {team_2_off_adj}")
            print(f"{team_1} averages {round(team_1_tempo)} possessions per game")
            print(f"{team_2} averages {round(team_2_tempo)} possessions per game")
            print(f"{team_1} is therefore expected to score {round((team_1_tempo)*(team_1_off_adj/100))} points")
            print(f"{team_2} is therefore expected to score {round((team_2_tempo)*(team_2_off_adj/100))} points")
        else:
            print(f"{team_1}'s expected efficiency is {team_1_off_adj}")
            print(f"{team_2}'s expected efficiency is {team_2_off_adj}")
            print(f"{team_1} averages {round(team_1_tempo)} possessions per game")
            print(f"{team_2} averages {round(team_2_tempo)} possessions per game")
            print(f"{team_1} is therefore expected to score {round((team_1_tempo)*(team_1_off_adj/100)+3.75)} points")
            print(f"{team_2} is therefore expected to score {round((team_2_tempo)*(team_2_off_adj/100))} points")
    if neutral:
        return(round((team_1_tempo)*(team_1_off_adj/100)),round((team_2_tempo)*(team_2_off_adj/100)))
    else:
        return(round((team_1_tempo)*(team_1_off_adj/100)+3.75),round((team_2_tempo)*(team_2_off_adj/100)))

In [87]:
get_expected_spread(sum_df,'Purdue','Indiana',neutral = False,verbose = True)

Purdue's expected efficiency is 112.85
Indiana's expected efficiency is 103.53
Purdue averages 66 possessions per game
Indiana averages 68 possessions per game
Purdue is therefore expected to score 79 points
Indiana is therefore expected to score 70 points


(79, 70)